In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
import tensorflow_hub as hub
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import numpy as np
import nltk
import re

In [5]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
data = pd.read_csv("C:\__DESKTOP\GitHub\Political-Sentiment-Analysis\word2vec\Twitter_Data.csv")

data = shuffle(data)
data.reset_index(inplace=True, drop=True)
data = data[data.category!= 0]
data = data.sort_values(by=['clean_text'])
data = data[:-1190]
data = shuffle(data)
data = data.head(30000)

In [45]:
X = []
for sen in data['clean_text']:
  X.append(sen)
Y = np.array((data['category']+1)/2,dtype=np.int64)

In [46]:
print(type(X))
print(type(X[0]))
print(X[0])
print(type(Y))
print(type(Y[0]))
print(Y[0])

<class 'list'>
<class 'str'>
india growing day day bjp saying continually that for nation first but for opposition they have only one word say modi hato first satta lalo second loot jee bhar third 
<class 'numpy.ndarray'>
<class 'numpy.int64'>
1


In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [49]:
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [50]:
def preprocess(text):
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
          tokens.append(token)
    return " ".join(tokens)

def encoding(tweet):
  tweet = preprocess(tweet)
  tokens = tweet.split(" ")
  embedding = []
  for token in tokens:
    embedding.append(embed([token]).numpy()[0])

  embedding = np.array(embedding)
  padded_tweet = padding(embedding)
  return padded_tweet

def padding(encoded_tweet):
  count = 70 - encoded_tweet.shape[0]
  pad = np.zeros((1,250))
  for i in range(count):
    encoded_tweet = np.concatenate((pad,encoded_tweet), axis=0)
  return encoded_tweet


def processed_tweets(tweets):
  x = []
  for tweet in tweets:
    enc = encoding(tweet)
    x.append(enc)
  x = np.array(x)
  return x

In [51]:
X_train = processed_tweets(X_train)
X_test = processed_tweets(X_test)

KeyboardInterrupt: 

In [10]:
print(type(X_train))
print(X_test.shape)
print(X_train.shape)

<class 'numpy.ndarray'>
(6000, 70, 250)
(24000, 70, 250)


In [14]:
tweet = ['Pained by the loss of lives due to a fire at a hospital in Bharuch. Condolences to the bereaved families.','To add further momentum to our strong strategic partnership, President Putin and I have agreed to establish a Ministerial Dialogue between our Foreign and Defence Ministers.']
x = processed_tweets(tweet)
y = np.array([0,1])
print(x)
print(type(x[0][0][0]))
print(x.shape)

[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [-0.10141002  0.04614043  0.07535188 ...  0.06751132 -0.12479866
   -0.0409786 ]
  [-0.04654795 -0.03045449  0.09535213 ...  0.1478346  -0.06534926
    0.02823413]
  [-0.14189312 -0.04154152  0.06161864 ...  0.02668094  0.03337354
    0.03328392]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [-0.1441164   0.00284827  0.02030733 ...  0.0143747   0.03983784
   -0.03091881]
  [ 0.03566772  0.00216552 -0.03347598 ...  0.11814095  0.03247234
    0.0506485 ]
  [ 0.00080403  0.08247177  0.02548021 ...  0.05351444 -0.0986348
   -0.01128561]]]
<class 'numpy.float64'>
(2, 70, 250)


In [11]:
clf = Sequential()

clf.add(LSTM(128, activation='relu', return_sequences=True))
clf.add(Dropout(0.2))

clf.add(LSTM(128, activation='relu'))
clf.add(Dropout(0.2))

clf.add(Dense(32, activation='relu'))
clf.add(Dropout(0.2))

clf.add(Dense(1,activation='sigmoid'))

clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
clf.fit(X_train, Y_train, validation_data=(X_test,Y_test), epochs=10)

In [ ]:
clf.predict(processed_tweets(tweet))